# Sentence Classification

## Background

In a scenario where an algorithm classifies sectors for different texts, it is important to measure its classification. This makes it possible to understand the worst assertiveness rate and propose solutions for improvement.
This project aims to develop a model that classifies different subjects for sentences and evaluates their quality.

The project was made possible according to the [proposed planning](../docs/planning.md)

## Measure of Success

As we want to understand which are the worst classifications, metrics such as Recall and Precision are able to correctly measure the correct classifications, in this case we will use F1-Score which harmonizes the metrics. In addition, we will analyze the confusion matrix and AUC Curve to identify the classifications for each label.

# 1.0 Imports

In [20]:
import warnings
import pandas                            as pd
import string
import re
import numpy                             as np

from collections import Counter
import seaborn                           as sns
import matplotlib.pyplot                 as plt

import nltk
from nltk.stem                          import WordNetLemmatizer
from nltk.corpus                        import stopwords, wordnet

from sklearn.preprocessing              import MultiLabelBinarizer
from sklearn.model_selection            import train_test_split
from sklearn.metrics                    import precision_score, recall_score, f1_score, confusion_matrix

from wordcloud                          import WordCloud

from keras.preprocessing.text           import Tokenizer
from keras.preprocessing.sequence       import pad_sequences
import tensorflow                       as tf
from tensorflow.keras                   import backend as K
from keras                              import saving
from tensorflow.keras.layers            import Input, SpatialDropout1D, Conv1D, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.models            import Model
from tensorflow.keras.regularizers      import l1_l2
from keras.callbacks                    import EarlyStopping
from keras.models                       import load_model

## 1.1 Load Data

In [21]:
df_raw = pd.read_csv("../data/01_raw/dataset.csv")

## 1.2 Helper Funtions

In [22]:
def notebook_settings():
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', 60)
    pd.set_option('display.float_format', lambda x: '%.3f' % x)

    plt.style.use('bmh')
    plt.rcParams['figure.figsize'] = [20, 12]
    plt.rcParams['font.size'] = 24
    sns.set()

    warnings.filterwarnings('ignore')
    return None


def lemmatize_word(word, lemmatizer):
    lemma = lemmatizer.lemmatize(word, pos=wordnet.VERB)
    return lemmatizer.lemmatize(lemma, pos=wordnet.NOUN)


def process_text(text, lemmatizer, portuguese_stopword):
    text = text.lower()

    text = ''.join([char for char in text if char not in string.punctuation])
    
    words = text.split()
    filtered_and_lemmatized_words = [lemmatize_word(word, lemmatizer) for word in words if word.lower() not in portuguese_stopword]
    text = ' '.join(filtered_and_lemmatized_words)
    
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text


@saving.register_keras_serializable()
def f2_score(y_true, y_pred):
    beta_squared = 4

    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=1)
    fp = K.sum(K.round(K.clip(y_pred - y_true, 0, 1)), axis=1)
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)), axis=1)

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())

    fbeta_score = (1 + beta_squared) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())
    return K.mean(fbeta_score)


def sample_prediction_multilabel(model, tokenizer, validation_data, original_data, mlb, MAX_SEQUENCE_LENGTH, num_samples=5, threshold=0.5):
    sample_indices = np.random.choice(validation_data.index, size=num_samples, replace=False)

    sample_texts_processed = validation_data.loc[sample_indices, 'sentence'].tolist()
    sample_texts_original = original_data.loc[sample_indices, 'sentence'].tolist()

    sample_true_categories = validation_data.loc[sample_indices, mlb.classes_].values

    sequences = tokenizer.texts_to_sequences(sample_texts_processed)
    data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    
    predictions = model.predict(data)
    
    predicted_categories = (predictions > threshold).astype(int)
    
    for i, (text, true_cats, pred_cats) in enumerate(zip(sample_texts_original, sample_true_categories, predicted_categories)):
        print('-' * 100)
        print(f"Sample {i+1}")
        print(f"Original Text: {text}")
        print(f"True Categories: {[mlb.classes_[i] for i, val in enumerate(true_cats) if val == 1]}")
        print(f"Predicted Categories: {[mlb.classes_[i] for i, val in enumerate(pred_cats) if val == 1]}")
    print('-' * 100)
    return None


def evaluate_metrics(y_true, y_pred):
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    return None


def plot_multilabel_confusion_matrix(y_true, y_pred, labels, figsize=(15, 10)):
    n_labels = y_true.shape[1]
    
    fig, axs = plt.subplots(int(np.ceil(n_labels / 3)), 3, figsize=figsize)
    axs = axs.flatten()

    for i in range(n_labels):
        cm = confusion_matrix(y_true[:, i], y_pred[:, i])
        
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axs[i])
        axs[i].set_title(labels[i])
        axs[i].set_xlabel('Predicted Label')
        axs[i].set_ylabel('True Label')

    for i in range(n_labels, len(axs)):
        axs[i].axis('off')

    plt.tight_layout()
    plt.show()

    
notebook_settings()

# 2.0 Data Descrition

In [23]:
data_description = df_raw.copy()

data_description.shape

(521, 2)

## 2.1 Check NaNs

In [24]:
data_description.isna().sum()

sentence    0
category    0
dtype: int64

## 2.2 Data Types

In [25]:
data_description.dtypes

sentence    object
category    object
dtype: object

- We don't have any visible problems with the data, so let's do some more in-depth analysis.

# 3.0 Analyse Data

In [26]:
data_analysis = data_description.copy()

In [27]:
data_analysis['category'].value_counts(normalize=True)

category
orgão público              0.269
educação                   0.205
indústrias                 0.171
varejo                     0.163
finanças                   0.104
finanças,varejo            0.025
educação,orgão público     0.017
indústrias,varejo          0.013
educação,indústrias        0.010
finanças,orgão público     0.008
finanças,indústrias        0.006
indústrias,orgão público   0.004
educação,finanças          0.004
varejo,indústrias          0.002
Name: proportion, dtype: float64

In [28]:
data_analysis['category'].value_counts()

category
orgão público               140
educação                    107
indústrias                   89
varejo                       85
finanças                     54
finanças,varejo              13
educação,orgão público        9
indústrias,varejo             7
educação,indústrias           5
finanças,orgão público        4
finanças,indústrias           3
indústrias,orgão público      2
educação,finanças             2
varejo,indústrias             1
Name: count, dtype: int64

- We have unbalanced data, which is to be expected, but we also have some classes such as 1 occurrence, which can get in the way of dividing up the data; if we need to improve the model, we can apply class balancing.

In [29]:
data_analysis['sentence_lenght'] = data_analysis['sentence'].apply(lambda x: len(str(x).split()))
data_analysis['sentence_lenght'].sort_values(ascending=False)

210    21
193    20
303    20
507    20
362    20
       ..
64      3
386     3
491     3
244     3
212     3
Name: sentence_lenght, Length: 521, dtype: int64

In [30]:
data_analysis.iloc[210, 0]

'Para se enquadrar nesta categoria é necessário apresentar o comprovante de produtor rural, o ITR – Imposto sobre propriedade Territorial Rural.'

- We don't have long sentences, which makes it easier for models not to have to maintain a long context.

## 3.1 Most Used Words

In [31]:
all_texts = ''.join(data_analysis['sentence'])

words = all_texts.split()

word_counts = Counter(words)
df_words = pd.DataFrame(word_counts.items(), columns=['word','count'])


df_words.sort_values(by='count', ascending=False).head(20)

,word,count
22,de,351
5,e,145
24,do,106
110,o,99
30,a,93
42,para,89
3,em,72
20,da,65
120,com,54
216,no,40


- We have a lot of `stopwords`, which is to be expected, so we'll deal with those cases.